In [1]:
import time
from ortools.linear_solver import pywraplp
import pandas as pd

# Crear el sportar olver
solver = pywraplp.Solver.CreateSolver('SCIP')
if not solver:
    print("El solver no está disponible.")
    exit()

# Ejercicio 1

# 1. Procesamiento de los datos

Leemos los archivos de ejemplo y nos guardamos sus datos en listas o matrices para luego utilizarlas en el modelo.

In [2]:
#Primero nos vamos a guardar las demandas.
# Inicializamos la lista donde se guardarán los valores de la segunda columna
demandas = {}
# Abrimos el archivo.txt en modo lectura
with open('oficinas.txt', 'r') as archivo:
    for iterador, linea in enumerate(archivo):  # iterador comenzará desde 0
        columnas = linea.split()
        demandas[iterador] = int(columnas[1])

# Leemos el archivo de distancias con pandas, asumiendo que está separado por tabulaciones (por defecto en read_csv)
distancias = pd.read_csv('distancias.txt', header=None, delimiter='\t')
# Eliminamos la última columna de la matriz de distancias
distancias = distancias.drop(distancias.columns[-1], axis=1)


# 2. Creación del modelo

creamos las variables donde

C: Son las centrales del modelo

O: Son las oficinas del modelo

d: Es una matriz con las distancias (en metros) de cada oficina a cada central

D: Demandas de las oficinas

In [3]:
d = distancias
O = range(len(demandas))
C = range(distancias.shape[1])

# Creamos las variables binarias

### Variables Binarias del Modelo
Para nuestro modelo, definimos las siguientes variables binarias:

$x_{ij} =
\begin{cases} 
1 & \text{si la oficina } i \text{ está atendida por la central } j \\
0 & \text{en caso contrario.}
\end{cases}
$


$
y_{j} =
\begin{cases}
1 & \text{si la central } j \text{ está activada} \\
0 & \text{en caso contrario.}
\end{cases}
$

$d_{ij}$: Distancia (en metros) entre la oficina $i$ y la central $j$.

$D$

$i \in O,  j \in C$


In [4]:
# Variables de decisión
# X[i][j] = 1 si la oficina i está atendida por la central j
x = {}
x = {(i, j): solver.BoolVar(f'x[{i},{j}]') for i in O for j in C}
# Y[j] = 1 si la central j está activada, 0 si no
y = {j: solver.BoolVar(f'y[{j}]') for j in C}
print(y)
print(x)

{0: y[0], 1: y[1], 2: y[2], 3: y[3], 4: y[4], 5: y[5], 6: y[6], 7: y[7], 8: y[8], 9: y[9]}
{(0, 0): x[0,0], (0, 1): x[0,1], (0, 2): x[0,2], (0, 3): x[0,3], (0, 4): x[0,4], (0, 5): x[0,5], (0, 6): x[0,6], (0, 7): x[0,7], (0, 8): x[0,8], (0, 9): x[0,9], (1, 0): x[1,0], (1, 1): x[1,1], (1, 2): x[1,2], (1, 3): x[1,3], (1, 4): x[1,4], (1, 5): x[1,5], (1, 6): x[1,6], (1, 7): x[1,7], (1, 8): x[1,8], (1, 9): x[1,9], (2, 0): x[2,0], (2, 1): x[2,1], (2, 2): x[2,2], (2, 3): x[2,3], (2, 4): x[2,4], (2, 5): x[2,5], (2, 6): x[2,6], (2, 7): x[2,7], (2, 8): x[2,8], (2, 9): x[2,9], (3, 0): x[3,0], (3, 1): x[3,1], (3, 2): x[3,2], (3, 3): x[3,3], (3, 4): x[3,4], (3, 5): x[3,5], (3, 6): x[3,6], (3, 7): x[3,7], (3, 8): x[3,8], (3, 9): x[3,9], (4, 0): x[4,0], (4, 1): x[4,1], (4, 2): x[4,2], (4, 3): x[4,3], (4, 4): x[4,4], (4, 5): x[4,5], (4, 6): x[4,6], (4, 7): x[4,7], (4, 8): x[4,8], (4, 9): x[4,9], (5, 0): x[5,0], (5, 1): x[5,1], (5, 2): x[5,2], (5, 3): x[5,3], (5, 4): x[5,4], (5, 5): x[5,5], (5, 6): x[5,

\begin{aligned}
\text{\Large Creamos nuestra función objetivo:}
\end{aligned}

\begin{aligned}
\text{Min} \sum_{i \in O} \sum_{j \in C} 0.017 \, d_{ij} \, x_{ij} + \sum_{j \in C} 5700 \, y_j
\end{aligned} 

In [5]:
coste_activacion = 5700  # Costo de activación de cada central
coste_distancia = 17/1000  # Costo por metro de distancia
capacidad_maxima_central = 15000  # Capacidad máxima de cada central
# Función objetivo: minimizar costos
# Costo total = Activación de las centrales + Costo por distancia
objetivo = solver.Objective()
for i in O:
    for j in C:
        # Costo de activar la central y el costo de distancia entre la oficina i y la central j
        objetivo.SetCoefficient(x[(i, j)], coste_distancia * d[j][i])  # distancias[i][j]
        
for j in C:
    # Costo de activar la central
    objetivo.SetCoefficient(y[j], coste_activacion)

objetivo.SetMinimization()


### Planteamos las restricciones de nuestro modelo

\begin{aligned}
\text{Restricción 1: Cada oficina debe ser atendida por exactamente una central}
\end{aligned}

\begin{aligned}
\sum_{j \in C} x_{ij} &= 1 \quad \forall i \in O
\end{aligned}

In [6]:
for i in O:
    solver.Add(solver.Sum(x[(i, j)] for j in C) == 1)

\begin{aligned}
\text{Restricción 2: Una central no puede atender más de su capacidad máxima}
\end{aligned}

\begin{aligned}
\sum_{i \in O} x_{ij} D_{i} &\leq 15000 \quad \forall j \in C
\end{aligned}

In [7]:
# Restricción de capacidad de las centrales: una central no puede atender más de su capacidad máxima
for j in C:
    solver.Add(solver.Sum(x[(i, j)] * demandas[i] for i in O) <= capacidad_maxima_central)

\begin{aligned}
\text{Restricción 3: Si una oficina es atendida por una central, entonces esa central debe estar activada}
\end{aligned}

\begin{aligned}
x_{ij} \leq y_j \quad \forall i \in O, \, \forall j \in C
\end{aligned}

In [8]:
# Restricción de que si una oficina es atendida por una central, entonces esa central debe estar activada
for i in O:
    for j in C:
        solver.Add(x[(i, j)] <= y[j])

# 3. Resolvemos el modelo

In [9]:
# Resolver el modelo
status = solver.Solve()

# Imprimir resultados
if status == pywraplp.Solver.OPTIMAL:
    print("Valor óptimo de la función objetivo:", solver.Objective().Value())
    print("Valores óptimos de las variables de decisión:")
    
    # Diccionario para almacenar cuántas oficinas atiende cada central
    oficinas_por_central = {j: 0 for j in C}

    for j in C:
        if y[j].solution_value() == 1:
            print(f"Central {j} está activada")
    
    for i in O:
        for j in C:
            if x[(i, j)].solution_value() == 1:
                print(f"Oficina {i} está atendida por la central {j}")
                oficinas_por_central[j] += 1  # Incrementar el contador para la central correspondiente

    # Mostrar el total de oficinas atendidas por cada central
    print("\nCantidad de oficinas atendidas por cada central:")
    for j in C:
        print(f"Central {j}: {oficinas_por_central[j]} oficinas")
else:
    print("No se encontró una solución óptima.")


Valor óptimo de la función objetivo: 34549.843
Valores óptimos de las variables de decisión:
Central 0 está activada
Central 1 está activada
Central 2 está activada
Central 4 está activada
Central 6 está activada
Central 8 está activada
Oficina 0 está atendida por la central 1
Oficina 1 está atendida por la central 1
Oficina 2 está atendida por la central 1
Oficina 3 está atendida por la central 6
Oficina 4 está atendida por la central 6
Oficina 5 está atendida por la central 6
Oficina 6 está atendida por la central 6
Oficina 7 está atendida por la central 4
Oficina 8 está atendida por la central 4
Oficina 9 está atendida por la central 2
Oficina 10 está atendida por la central 2
Oficina 11 está atendida por la central 8
Oficina 12 está atendida por la central 8
Oficina 13 está atendida por la central 1
Oficina 14 está atendida por la central 0
Oficina 15 está atendida por la central 1
Oficina 16 está atendida por la central 4
Oficina 17 está atendida por la central 2
Oficina 18 está a

# Ejercicio 2: Añadir nuevas restricciones

\begin{aligned}
\text{Restricción nueva: cada central operativa no puede atender a mas de 10 oficinas}
\end{aligned}

\begin{aligned}
\sum_{i \in O} x_{ij} &\leq 10 \quad \forall j \in C
\end{aligned}

In [12]:
# Restricción de maxima cantidad de clientes por central
for j in C:
    solver.Add(solver.Sum(x[(i, j)] for i in O) <= 10)

In [13]:
# Resolver el modelo
status = solver.Solve()

# Imprimir resultados
if status == pywraplp.Solver.OPTIMAL:
    print("Valor óptimo de la función objetivo:", solver.Objective().Value())
    print("Valores óptimos de las variables de decisión:")
    
    # Diccionario para almacenar cuántas oficinas atiende cada central
    oficinas_por_central = {j: 0 for j in C}

    for j in C:
        if y[j].solution_value() == 1:
            print(f"Central {j} está activada")
    
    for i in O:
        for j in C:
            if x[(i, j)].solution_value() == 1:
                print(f"Oficina {i} está atendida por la central {j}")
                oficinas_por_central[j] += 1  # Incrementar el contador para la central correspondiente

    # Mostrar el total de oficinas atendidas por cada central
    print("\nCantidad de oficinas atendidas por cada central:")
    for j in C:
        print(f"Central {j}: {oficinas_por_central[j]} oficinas")
else:
    print("No se encontró una solución óptima.")


Valor óptimo de la función objetivo: 34549.843
Valores óptimos de las variables de decisión:
Central 0 está activada
Central 1 está activada
Central 2 está activada
Central 4 está activada
Central 6 está activada
Central 8 está activada
Oficina 0 está atendida por la central 1
Oficina 1 está atendida por la central 1
Oficina 2 está atendida por la central 1
Oficina 3 está atendida por la central 6
Oficina 4 está atendida por la central 6
Oficina 5 está atendida por la central 6
Oficina 6 está atendida por la central 6
Oficina 7 está atendida por la central 4
Oficina 8 está atendida por la central 4
Oficina 9 está atendida por la central 2
Oficina 10 está atendida por la central 2
Oficina 11 está atendida por la central 8
Oficina 12 está atendida por la central 8
Oficina 13 está atendida por la central 1
Oficina 14 está atendida por la central 0
Oficina 15 está atendida por la central 1
Oficina 16 está atendida por la central 4
Oficina 17 está atendida por la central 2
Oficina 18 está a

# Ejercicio 3


Para este ejercicio decidimos plantear otro modelo. 
La idea es minimizar la capacidad de cada central tal que el modelo original siga teniendo una solución factible

In [14]:
from ortools.linear_solver import pywraplp
import pandas as pd

# Crear el sportar olver
solver2 = pywraplp.Solver.CreateSolver('SCIP')
if not solver2:
    print("El solver no está disponible.")
    exit()

 ### Variables Binarias del Modelo
Para nuestro modelo, definimos las siguientes variables binarias:

$x_{ij} =
\begin{cases} 
1 & \text{si la oficina } i \text{ está atendida por la central } j \\
0 & \text{en caso contrario.}
\end{cases}
$


$
y_{j} =
\begin{cases}
1 & \text{si la central } j \text{ está activada} \\
0 & \text{en caso contrario.}
\end{cases}
$


$i \in O,  j \in C$


In [15]:
# Variables de decisión
# X[i][j] = 1 si la oficina i está atendida por la central j
x = {}
x = {(i, j): solver2.BoolVar(f'x[{i},{j}]') for i in O for j in C}
# Y[j] = 1 si la central j está activada, 0 si no
y = {j: solver2.BoolVar(f'y[{j}]') for j in C}


Capacidad es un número entero que tiene que ser minimizado, entonces planteamos: 

\begin{aligned}
\text{\Large Función objetivo de minimización de la capacidad:}
\end{aligned}

\begin{aligned}
\text{Min}\quad capacidad
\end{aligned} 

In [16]:
# Crear la variable de decisión (entera)
capacidad = solver2.IntVar(0, solver2.infinity(), 'x')  # x >= 0, sin límite superior

# Definir la función objetivo: minimizar x
solver2.Minimize(capacidad)

\begin{aligned}
\text{Restricción 1: Cada oficina debe ser atendida por exactamente una central}
\end{aligned}

\begin{aligned}
\sum_{j \in C} x_{ij} &= 1 \quad \forall i \in O
\end{aligned}

In [17]:
for i in O:
    solver2.Add(solver2.Sum(x[(i, j)] for j in C) == 1)

\begin{aligned}
\text{Restricción 2: Una central no puede atender más de su capacidad máxima}
\end{aligned}

\begin{aligned}
\sum_{i \in O} x_{ij} D_{i} &\leq capacidad \quad \forall j \in C
\end{aligned}

In [18]:
# Restricción de capacidad de las centrales: una central no puede atender más de su capacidad máxima
for j in C:
    solver2.Add(solver2.Sum(x[(i, j)] * demandas[i] for i in O) <= capacidad)

\begin{aligned}
\text{Restricción 3: Si una oficina es atendida por una central, entonces esa central debe estar activada}
\end{aligned}

\begin{aligned}
x_{ij} \leq y_j \quad \forall i \in O, \, \forall j \in C
\end{aligned}

In [19]:
# Restricción de que si una oficina es atendida por una central, entonces esa central debe estar activada
for i in O:
    for j in C:
        solver2.Add(x[(i, j)] <= y[j])

Finalmente resolvemos el modelo

In [20]:
# Resolver el modelo
status = solver2.Solve()

# Imprimir resultados
if status == pywraplp.Solver.OPTIMAL:
    capacidad_maxima_central = solver2.Objective().Value()
    print("Valor óptimo de la función objetivo:", capacidad_maxima_central)

else:
    print("No se encontró una solución óptima.")


Valor óptimo de la función objetivo: 8800.0


# Ejercicio 4

Primero generamos aleatoriamente los 7 archivos que vamos a testear

In [7]:
import numpy as np
import os

# Asegúrate de que la carpeta existe o créala si no existe
output_folder = "txts_ejercicio4"
os.makedirs(output_folder, exist_ok=True)

# Generar valores aleatorios para la cantidad de oficinas y centrales
oficinas_random = [50, 100, 50, 150, 150, 75, 50]
centrales_random = [10, 10, 25, 18, 18, 13, 22]

print("Valores de oficinas generados:", oficinas_random)
print("Valores de centrales generados:", centrales_random)

# Función para generar distancias aleatorias
def generar_matriz_distancias(num_oficinas, num_centrales, rango_distancias=(200, 4000)):
    # Generar matriz de distancias aleatorias
    return np.random.randint(rango_distancias[0], rango_distancias[1], size=(num_oficinas, num_centrales))

# Generar y guardar los 7 archivos
for i in range(7):
    num_oficinas = oficinas_random[i]  # Cantidad de oficinas para este archivo
    num_centrales = centrales_random[i]  # Cantidad de centrales para este archivo
    
    # Generar matriz de distancias
    matriz_distancias = generar_matriz_distancias(num_oficinas, num_centrales)
    
    # Generar columna de demandas aleatorias para las oficinas
    demandas = np.random.randint(200, 3000, size=num_oficinas).reshape(-1, 1)  
    
    # Concatenar la columna de demandas a la matriz de distancias
    matriz_final = np.hstack([matriz_distancias, demandas])  # Añadir la columna de demandas
    
    # Guardar en un archivo de distancias y demandas en la carpeta
    filename = os.path.join(output_folder, f'distancias_oficinas_{num_oficinas}_centrales_{num_centrales}_{i}.txt')
    np.savetxt(filename, matriz_final, fmt='%d', delimiter='\t')
    print(f'Archivo generado: {filename}')
    
    # Crear archivo de oficinas con solo el número de oficina y la demanda
    oficinas_demanda = np.column_stack((np.arange(1, num_oficinas + 1), demandas.flatten()))
    oficinas_filename = os.path.join(output_folder, f'oficinas_demanda_{num_oficinas}_{i}.txt')
    np.savetxt(oficinas_filename, oficinas_demanda, fmt='%d\t%d', delimiter='\t')
    print(f'Archivo de demandas generado: {oficinas_filename}')


Valores de oficinas generados: [50, 100, 50, 150, 150, 75, 50]
Valores de centrales generados: [10, 10, 25, 18, 18, 13, 22]
Archivo generado: txts_ejercicio4/distancias_oficinas_50_centrales_10_0.txt
Archivo de demandas generado: txts_ejercicio4/oficinas_demanda_50_0.txt
Archivo generado: txts_ejercicio4/distancias_oficinas_100_centrales_10_1.txt
Archivo de demandas generado: txts_ejercicio4/oficinas_demanda_100_1.txt
Archivo generado: txts_ejercicio4/distancias_oficinas_50_centrales_25_2.txt
Archivo de demandas generado: txts_ejercicio4/oficinas_demanda_50_2.txt
Archivo generado: txts_ejercicio4/distancias_oficinas_150_centrales_18_3.txt
Archivo de demandas generado: txts_ejercicio4/oficinas_demanda_150_3.txt
Archivo generado: txts_ejercicio4/distancias_oficinas_150_centrales_18_4.txt
Archivo de demandas generado: txts_ejercicio4/oficinas_demanda_150_4.txt
Archivo generado: txts_ejercicio4/distancias_oficinas_75_centrales_13_5.txt
Archivo de demandas generado: txts_ejercicio4/oficinas

In [2]:
import pandas as pd
import os
import time
from ortools.linear_solver import pywraplp
import pandas as pd

def leer_archivo(oficinas, distancias):
    demandas = {}
    # Cambiamos el directorio de trabajo si es necesario
    carpeta = 'txts_ejercicio4'  # Asegúrate de tener la ruta correcta
    ruta_oficinas = os.path.join(carpeta, oficinas)  # Crear la ruta completa al archivo de oficinas
    ruta_distancias = os.path.join(carpeta, distancias)  # Crear la ruta completa al archivo de distancias
    
    # Abrimos el archivo de oficinas en modo lectura
    with open(ruta_oficinas, 'r') as archivo:
        for iterador, linea in enumerate(archivo):  # iterador comenzará desde 0
            columnas = linea.split()
            demandas[iterador] = int(columnas[1])

    # Leemos el archivo de distancias con pandas, asumiendo que está separado por tabulaciones
    distancias_df = pd.read_csv(ruta_distancias, header=None, delimiter='\t')
    
    # Eliminamos la última columna de la matriz de distancias
    distancias_df = distancias_df.drop(distancias_df.columns[-1], axis=1)
    
    return distancias_df, demandas

# Ejemplo de cómo usar la función

In [4]:
from ortools.linear_solver import pywraplp
import numpy as np

coste_activacion = 5700  # Costo de activación de cada central
coste_distancia = 17 / 1000  # Costo por metro de distancia
capacidad_maxima_central = 15000  # Capacidad máxima de cada central

def modelo(C, O, d, demandas):
    # Crear un nuevo solver cada vez que resuelvas el modelo
    solver = pywraplp.Solver.CreateSolver('SCIP')
    if not solver:
        print("No se pudo crear el solver.")
        return
    
    # X[i][j] = 1 si la oficina i está atendida por la central j
    x = {(i, j): solver.BoolVar(f'x[{i},{j}]') for i in O for j in C}
    
    # Y[j] = 1 si la central j está activada, 0 si no
    y = {j: solver.BoolVar(f'y[{j}]') for j in C}
    
    # Crear el objetivo del solver
    objetivo = solver.Objective()

    # Planteamos la función objetivo
    for i in O:
        for j in C:
            # Costo de activar la central y el costo de distancia entre la oficina i y la central j
            objetivo.SetCoefficient(x[(i, j)], coste_distancia * d[j][i])  # distancias[i][j]
    
    for j in C:
        # Costo de activar la central
        objetivo.SetCoefficient(y[j], coste_activacion)

    objetivo.SetMinimization()

    # Planteamos las restricciones del modelo

    # Cada oficina debe ser atendida por exactamente una central
    for i in O:
        solver.Add(solver.Sum(x[(i, j)] for j in C) == 1)
    
    # Restricción de capacidad de las centrales: una central no puede atender más de su capacidad máxima
    for j in C:
        solver.Add(solver.Sum(x[(i, j)] * demandas[i] for i in O) <= capacidad_maxima_central)
    
    # Restricción de que si una oficina es atendida por una central, entonces esa central debe estar activada
    for i in O:
        for j in C:
            solver.Add(x[(i, j)] <= y[j])

    # Resolver el modelo
    status = solver.Solve()

    # Imprimir resultados
    if status == pywraplp.Solver.OPTIMAL:
        print("Valor óptimo de la función objetivo:", solver.Objective().Value())
        print("Valores óptimos de las variables de decisión:")
        for j in C:
            if y[j].solution_value() == 1:
                print(f"Central {j} está activada")
        
        # Crear un diccionario para las centrales y sus oficinas
        centrales = {}
        for j in C:
            centrales[j+1] = []
            for i in O:
                if x[(i, j)].solution_value() == 1:
                    centrales[j+1].append(i+1)
        
        # Mostrar qué oficinas están siendo atendidas por cada central
        for clave in centrales.keys():
            if len(centrales[clave]) != 0:
                print(f"La central {clave} atiende a las oficinas {centrales[clave]}")
    else:
        print("No se encontró una solución óptima.")


Ejecutamos el modelo para el archivo con 150 oficinas y 10 centrales 

In [36]:
distancias, demandas = leer_archivo('oficinas_demanda_100_1.txt', 'distancias_oficinas_100_centrales_10_1.txt')
d = distancias
O = range(len(demandas))
C = range(distancias.shape[1])
print("Modelo con 100 oficinas y 10 centrales")
distancias_promedio_modelo = np.mean(distancias)
demandas_promedio = sum(demandas.values()) / len(demandas)
print("Las distancias promedio son: ", distancias_promedio_modelo)
print("Las demandas promedio son: ", demandas_promedio)

start_time = time.time()
modelo(C,O,distancias,demandas)
# Medir el tiempo después de ejecutar el modelo
end_time = time.time()

# Calcular el tiempo que tardó en ejecutarse el modelo
execution_time = end_time - start_time
# Imprimir el tiempo de ejecución

print(f"El modelo tardó {execution_time:.4f} segundos en ejecutarse.")


Modelo con 100 oficinas y 10 centrales
Las distancias promedio son:  2136.12
Las demandas promedio son:  1611.49
No se encontró una solución óptima.
El modelo tardó 0.0490 segundos en ejecutarse.


Ejecutamos el modelo para el archivo con 50 oficinas y 10 centrales 

In [37]:
distancias, demandas = leer_archivo('oficinas_demanda_50_0.txt', 'distancias_oficinas_50_centrales_10_0.txt')
d = distancias
O = range(len(demandas))
C = range(distancias.shape[1])
# Medir el tiempo antes de ejecutar el modelo
print("Modelo con 50 oficinas y 10 centrales")
distancias_promedio_modelo = np.mean(distancias)
demandas_promedio = sum(demandas.values()) / len(demandas)
print("Las distancias promedio son: ", distancias_promedio_modelo)
print("Las demandas promedio son: ", demandas_promedio)

start_time = time.time()
modelo(C,O,distancias,demandas)
# Medir el tiempo después de ejecutar el modelo
end_time = time.time()
# Calcular el tiempo que tardó en ejecutarse el modelo
execution_time = end_time - start_time
# Imprimir el tiempo de ejecución
execution_time = execution_time
print(f"El modelo tardó {execution_time:.4f} segundos en ejecutarse.")


Modelo con 50 oficinas y 10 centrales
Las distancias promedio son:  2095.68
Las demandas promedio son:  1562.84
Valor óptimo de la función objetivo: 34778.305999999524
Valores óptimos de las variables de decisión:
Central 0 está activada
Central 1 está activada
Central 4 está activada
Central 6 está activada
Central 7 está activada
Central 8 está activada
La central 1 atiende a las oficinas [12, 15, 27, 29, 34, 38, 42, 44, 48, 50]
La central 2 atiende a las oficinas [1, 3, 5, 14, 19, 21, 30, 31, 49]
La central 5 atiende a las oficinas [2, 6, 7, 26, 33, 43, 45, 46]
La central 7 atiende a las oficinas [4, 8, 10, 16, 17, 18, 25, 28, 37]
La central 8 atiende a las oficinas [9, 32, 39, 40, 41, 47]
La central 9 atiende a las oficinas [11, 13, 20, 22, 23, 24, 35, 36]
El modelo tardó 0.7453 segundos en ejecutarse.


Ejecutamos el modelo para 75 oficinas y 13 centrales

In [38]:

distancias, demandas = leer_archivo('oficinas_demanda_75_5.txt', 'distancias_oficinas_75_centrales_13_5.txt')
d = distancias
O = range(len(demandas))
C = range(distancias.shape[1])
print("Modelo con 75 oficinas y 13 centrales")
distancias_promedio_modelo = np.mean(distancias)
demandas_promedio = sum(demandas.values()) / len(demandas)
print("Las distancias promedio son: ", distancias_promedio_modelo)
print("Las demandas promedio son: ", demandas_promedio)

start_time = time.time()
modelo(C,O,distancias,demandas)
end_time = time.time()
# Calcular el tiempo que tardó en ejecutarse el modelo
execution_time = end_time - start_time
# Imprimir el tiempo de ejecución
execution_time = execution_time
print(f"El modelo tardó {execution_time:.4f} segundos en ejecutarse.")


Modelo con 75 oficinas y 13 centrales
Las distancias promedio son:  2117.2851282051283
Las demandas promedio son:  1574.64
Valor óptimo de la función objetivo: 46315.785
Valores óptimos de las variables de decisión:
Central 3 está activada
Central 4 está activada
Central 5 está activada
Central 6 está activada
Central 7 está activada
Central 10 está activada
Central 11 está activada
Central 12 está activada
La central 4 atiende a las oficinas [8, 12, 13, 21, 38, 44, 51]
La central 5 atiende a las oficinas [3, 5, 17, 22, 32, 33, 35, 39, 40, 63, 69]
La central 6 atiende a las oficinas [24, 25, 41, 43, 45, 53, 56, 58, 71]
La central 7 atiende a las oficinas [4, 16, 19, 27, 30, 46, 55, 62, 64, 65, 74]
La central 8 atiende a las oficinas [2, 11, 18, 49, 50, 52, 66, 70, 72]
La central 11 atiende a las oficinas [6, 7, 14, 34, 37, 47, 68]
La central 12 atiende a las oficinas [20, 23, 29, 31, 54, 57, 59, 61, 67, 75]
La central 13 atiende a las oficinas [1, 9, 10, 15, 26, 28, 36, 42, 48, 60, 73]

Ejecutamos el modelo para el archivo que tiene 150 oficinas y 18 centrales

In [34]:

distancias, demandas = leer_archivo('oficinas_demanda_150_3.txt', 'distancias_oficinas_150_centrales_18_3.txt')
d = distancias
O = range(len(demandas))
C = range(distancias.shape[1])

print("Modelo con 150 oficinas y 18 centrales")
distancias_promedio_modelo = np.mean(distancias)
demandas_promedio = sum(demandas.values()) / len(demandas)
print("Las distancias promedio son: ", distancias_promedio_modelo)
print("Las demandas promedio son: ", demandas_promedio)


start_time = time.time()
modelo(C,O,distancias,demandas)
end_time = time.time()
# Calcular el tiempo que tardó en ejecutarse el modelo
execution_time = end_time - start_time
# Imprimir el tiempo de ejecución
execution_time = execution_time
print(f"El modelo tardó {execution_time:.4f} segundos en ejecutarse.")


Modelo con 150 oficinas y 18 centrales
Las distancias promedio son:  2124.485925925926
Las demandas promedio son:  1522.1
Valor óptimo de la función objetivo: 92238.7
Valores óptimos de las variables de decisión:
Central 0 está activada
Central 1 está activada
Central 2 está activada
Central 3 está activada
Central 4 está activada
Central 5 está activada
Central 7 está activada
Central 8 está activada
Central 10 está activada
Central 11 está activada
Central 12 está activada
Central 13 está activada
Central 14 está activada
Central 15 está activada
Central 16 está activada
Central 17 está activada
La central 1 atiende a las oficinas [38, 44, 45, 57, 65, 112, 116, 118, 132]
La central 2 atiende a las oficinas [3, 31, 36, 77, 98, 102, 131, 134]
La central 3 atiende a las oficinas [4, 5, 28, 49, 82, 120, 122, 125, 148]
La central 4 atiende a las oficinas [2, 25, 30, 40, 46, 60, 72, 90, 103]
La central 5 atiende a las oficinas [8, 10, 22, 23, 88, 101, 104, 115, 127, 142, 149]
La central 6 

Ejecutamos el modelo para el archivo que tiene 150 oficinas y 18 centrales

In [35]:

distancias, demandas = leer_archivo('oficinas_demanda_150_4.txt', 'distancias_oficinas_150_centrales_18_4.txt')

print("Modelo con 150 oficinas y 18 centrales")
distancias_promedio_modelo = np.mean(distancias)
demandas_promedio = sum(demandas.values()) / len(demandas)
print("Las distancias promedio son: ", distancias_promedio_modelo)
print("Las demandas promedio son: ", demandas_promedio)

d = distancias
O = range(len(demandas))
C = range(distancias.shape[1])

start_time = time.time()
modelo(C,O,distancias,demandas)
end_time = time.time()
# Calcular el tiempo que tardó en ejecutarse el modelo
execution_time = end_time - start_time
# Imprimir el tiempo de ejecución
execution_time = execution_time
print(f"El modelo tardó {execution_time:.4f} segundos en ejecutarse.")


Modelo con 150 oficinas y 18 centrales
Las distancias promedio son:  2082.495185185185
Las demandas promedio son:  1705.0666666666666
Valor óptimo de la función objetivo: 103646.724
Valores óptimos de las variables de decisión:
Central 0 está activada
Central 1 está activada
Central 2 está activada
Central 3 está activada
Central 4 está activada
Central 5 está activada
Central 6 está activada
Central 7 está activada
Central 8 está activada
Central 9 está activada
Central 10 está activada
Central 11 está activada
Central 12 está activada
Central 13 está activada
Central 14 está activada
Central 15 está activada
Central 16 está activada
Central 17 está activada
La central 1 atiende a las oficinas [17, 18, 29, 40, 74, 76, 107]
La central 2 atiende a las oficinas [24, 33, 55, 71, 117, 125, 145]
La central 3 atiende a las oficinas [63, 66, 81, 83, 104, 105, 131]
La central 4 atiende a las oficinas [2, 5, 6, 75, 84, 100, 114, 138, 139]
La central 5 atiende a las oficinas [26, 51, 56, 61, 82,

Ejecutamos el modelo para el archivo con 50 oficinas y 20 centrales

In [8]:
distancias, demandas = leer_archivo('oficinas_demanda_50_6.txt', 'distancias_oficinas_50_centrales_22_6.txt')

distancias_promedio_modelo = np.mean(distancias)
demandas_promedio = sum(demandas.values()) / len(demandas)
print("Las distancias promedio son: ", distancias_promedio_modelo)
print("Las demandas promedio son: ", demandas_promedio)

d = distancias
O = range(len(demandas))
C = range(distancias.shape[1])
print("Modelo con 50 oficinas y 22 centrales")
start_time = time.time()
modelo(C,O,distancias,demandas)
end_time = time.time()
# Calcular el tiempo que tardó en ejecutarse el modelo
execution_time = end_time - start_time
# Imprimir el tiempo de ejecución
execution_time = execution_time
print(f"El modelo tardó {execution_time:.4f} segundos en ejecutarse.")


Las distancias promedio son:  2062.4445454545453
Las demandas promedio son:  1575.16
Modelo con 50 oficinas y 22 centrales
Valor óptimo de la función objetivo: 34654.95399999999
Valores óptimos de las variables de decisión:
Central 2 está activada
Central 9 está activada
Central 12 está activada
Central 14 está activada
Central 17 está activada
Central 21 está activada
La central 3 atiende a las oficinas [1, 3, 6, 11, 13, 23, 29, 35, 37, 42]
La central 10 atiende a las oficinas [2, 4, 14, 17, 31, 32, 40]
La central 13 atiende a las oficinas [9, 10, 16, 21, 26, 30, 33, 39, 44]
La central 15 atiende a las oficinas [12, 15, 19, 25, 27, 41, 47]
La central 18 atiende a las oficinas [5, 18, 20, 28, 34, 38, 43]
La central 22 atiende a las oficinas [7, 8, 22, 24, 36, 45, 46, 48, 49, 50]
El modelo tardó 6287.9814 segundos en ejecutarse.


In [ ]:

distancias, demandas = leer_archivo('oficinas_demanda_50_2.txt', 'distancias_oficinas_50_centrales_25_2.txt')

distancias_promedio_modelo = np.mean(distancias)
demandas_promedio = sum(demandas.values()) / len(demandas)
print("Las distancias promedio son: ", distancias_promedio_modelo)
print("Las demandas promedio son: ", demandas_promedio)
d = distancias
O = range(len(demandas))
C = range(distancias.shape[1])

print("Modelo con 50 oficinas y 25 centrales")
start_time = time.time()
modelo(C,O,distancias,demandas)
end_time = time.time()
# Calcular el tiempo que tardó en ejecutarse el modelo
execution_time = end_time - start_time
# Imprimir el tiempo de ejecución
execution_time = execution_time
print(f"El modelo tardó {execution_time:.4f} segundos en ejecutarse.")


Las distancias promedio son:  2080.4696
Las demandas promedio son:  1668.28
Modelo con 50 oficinas y 25 centrales
pressed CTRL-C 1 times (5 times for forcing termination)
